In [ ]:
#default_exp models

# Convolutional block

In [ ]:
#exporti
import torch
import torch.nn as nn

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
#export
class ConvolutionalBlock(nn.Module):
    """
    This class defines a convolutional block that can be used for the construction of convolutional neural networks (CNNs).
    """
    def __init__(
        self,
        dimensions:int, # The number of dimensions to consider. Possible options are 2 and 3.
        in_channels:int, # The number of input channels.
        out_channels:int, # The number of output channels.
        normalization:str=None, # The type of normalization to use. Possible options include "batch", "layer" and "instance".
        kernel_size:int=3, # The size of the convolutional kernel.
        activation:str='ReLU', # The activation function that should be used.
        preactivation:bool=False, # Whether to use preactivations.
        use_padding:bool=True, # Whether to use padding.
        padding_mode:str='zeros', # The type of padding to use.
        dilation:bool=None, # The amount of dilation that should be used.
        dropout:float=0, # The dropout rate.
    ):
        super().__init__()
        self.dimensions = dimensions
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalization = normalization
        self.kernel_size = kernel_size
        self.activation = activation
        self.preactivation = preactivation
        self.use_padding = use_padding
        self.padding_mode = padding_mode
        self.dropout = dropout

        if dilation is None:
            self.dilation = 1

        self._set_padding()

        self._set_conv_layer()
        self._set_norm_layer()
        self._set_activation_layer()
        self._set_dropout_layer()

        self.block = self._create_sequential_block_from_layers()


    def forward(self, 
                x:torch.Tensor # The input to the convolutional block.
               ):
        """
        The forward pass of the convolutional block. Returns a `torch.Tensor`.
        """
        return self.block(x)


    def _set_padding(self):
        if self.use_padding:
            total_padding = self.dilation * (self.kernel_size - 1)
            self.padding = total_padding // 2
        else:
            self.padding = 0


    def _set_conv_layer(self):
        conv_class = getattr(nn, f'Conv{self.dimensions}d')
        no_bias = not self.preactivation and (self.normalization is not None)

        self.conv_layer = conv_class(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            padding=self.padding,
            padding_mode=self.padding_mode,
            dilation=self.dilation,
            bias=not no_bias
        )


    def _set_norm_layer(self):
        if self.normalization is None:
            self.norm_layer = None
        else:
            num_features = self.in_channels if self.preactivation else self.out_channels
            norm_class = getattr(nn, f'{self.normalization.capitalize()}Norm{self.dimensions}d')
            self.norm_layer = norm_class(num_features)


    def _set_activation_layer(self):
        if self.activation is None:
            self.activation_layer = None
        elif type(self.activation) == str:
            self.activation_layer = getattr(nn, self.activation)()
        else:
            self.activation_layer = self.activation


    def _set_dropout_layer(self):
        if self.dropout == 0 or self.dropout is None:
            self.dropout_layer = None
        else:
            dropout_class = getattr(nn, f'Dropout{self.dimensions}d')
            self.dropout_layer = dropout_class(p=self.dropout)


    def _create_sequential_block_from_layers(self):
        block = nn.ModuleList()

        if self.preactivation:
            self._add_if_not_none(block, self.norm_layer)
            self._add_if_not_none(block, self.activation_layer)
            self._add_if_not_none(block, self.conv_layer)
        else:
            self._add_if_not_none(block, self.conv_layer)
            self._add_if_not_none(block, self.norm_layer)
            self._add_if_not_none(block, self.activation_layer)

        self._add_if_not_none(block, self.dropout_layer)
        return nn.Sequential(*block)


    @staticmethod
    def _add_if_not_none(module_list, module):
        if module is not None:
            module_list.append(module)

In [ ]:
show_doc(ConvolutionalBlock.forward)

<h4 id="ConvolutionalBlock.forward" class="doc_header"><code>ConvolutionalBlock.forward</code><a href="__main__.py#L45" class="source_link" style="float:right">[source]</a></h4>

> <code>ConvolutionalBlock.forward</code>(**`x`**:`Tensor`)

The forward pass of the convolutional block. Returns a `torch.Tensor`.

||Type|Default|Details|
|---|---|---|---|
|**`x`**|`Tensor`||The input to the convolutional block.|


In [ ]:
%%time
#hide

def test_that_we_can_instanciate():
    topo_solver = ConvolutionalBlock(3, 1, 1)


test_that_we_can_instanciate()

CPU times: user 7.38 ms, sys: 2.66 ms, total: 10 ms
Wall time: 29.2 ms
